## ДЗ №3: Рекуррентные нейронные сети.  Обработка естественного языка

### Пожалуйста, заполните имя

In [ ]:
STUDENT_NAME = "FirstName LastName" # For example, Fedor Petriaikin

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch import optim
import torch
from torch import nn
import pandas
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset, TensorDataset
%matplotlib inline

**Не удаляйте клетки из этого jupyter-notebok! Это затруднит проверку!**

В этом задании вам предстоит попробовать различные рекуррентные архитектуры для предсказания временного ряда и написания простейшего чат-бота.


### Часть 1. Предсказание временного ряда

а) Напишите свою реализацию LSTM
<img src="https://cdn-images-1.medium.com/max/1600/1*hl9UVtgIcQkDIGD8VFykdw.png" width="640">

In [ ]:
# Кстати, на вход в pytorch-реализацию можно подавать последовательности разной длины.
# См. torch.nn.utils.rnn.pack_padded_sequence() и torch.nn.utils.rnn.pack_sequence()
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        ## YOUR CODE HERE
        pass
    
    # Пропускает через себя последовательность и выдает последний выход
    # X: (seq_len, batch_size, input_size)
    # hidden = (hidden, C): (batch_size, hidden_size)x2 - чтобы начать предсказания с какого-то момента
    # Если None, инициализировать нулями
    # Возвращает out, h, c, где (h, c) - выход и скрытое состояние после последнего элемента
    # out - (seq_len, batch, hidden_size) - выходы каждого слоя
    def forward(self, X, hidden=None):
        pass

б) Напишите код обучения LSTM на представленных данных

In [ ]:
# График (*)
%matplotlib inline
dataset = pandas.read_csv('international-airline-passengers.csv', usecols=[1], delimiter=";", engine='python', skipfooter=3)
plt.plot(dataset)
plt.show()

In [ ]:
# подготовливаем данные
dataset = dataset.values
dataset = dataset.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
# разбиваем данные на train / test
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

In [ ]:
# Целевая переменная - сдвинутая последовательность значений входов
# look_back - на сколько смещен test относительно train (seq_len)
# look_forward - сколько нужно предсказать
def create_dataset(dataset, look_back=1, look_forward=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1-look_forward):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back : (i + look_back + look_forward), 0])

    return np.array(dataX), np.array(dataY)

In [ ]:
# Пример подготовленных данных
look_back = 5
look_forward=1
trainX, trainY = create_dataset(train, look_back, look_forward)
testX, testY = create_dataset(test, look_back, look_forward)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
# Берет данные из dataloader, сам их сдвигает, формируя обучающую выборку
# В качестве функции ошибок возьмите MSELoss, оптимизатор - Adagrad
# В функцию можно добавить дополнительные параметры (например, max_epochs, условия останова...)
def TrainModel(model, dataloader, look_back=1, look_forward=1):
    ## YOUR CODE HERE
    pass

в) Попробуйте обучить сеть с разными параметрами look_back, look_forward. Также попробуйте использовать различные hidden_size. Выведите предсказания на график (\*). Сделайте вывод.  
*Обратите внимание*: "Классический", однонаправленный LSTM имеет размерность выхода (seq_len, batch, hidden_size). Но иногда нужно "изменить" размерность выхода - для этого можно обучить Linear: hidden_size x need_size  

г) Сравните свой результат и результат с использованием nn.LSTM (на аналогичном числе эпох, hidden_size и т.п.)  

д) Попробуйте использовать nn.GRU. Сравните результаты

In [ ]:
## YOUR CODE HERE

### Часть 2. RNN и обработка текстов

В этом задании мы будем использовать файл author-quote.txt

In [ ]:
np.random.seed(777)
torch.random.manual_seed(777)

In [ ]:
lines = [x.split('\t')[1].strip() for x in open('author-quote.txt').readlines()]
lines[0]

In [ ]:
lines_len = []
for l in map(len, lines):
    lines_len.append(l)

plt.hist(lines_len, bins=100);

In [ ]:
lines = [l for l in filter(lambda x: len(x) <= 50, lines)]

In [ ]:
class Vocabulary:
    def __init__(self, sentences):
        all_characters = set()
        for line in sentences:
            all_characters |= set(line)
        all_characters = list(all_characters)+['<eos>', '<go>']
        self.char_to_id = {x[1]:x[0] for x in enumerate(all_characters)}
        self.id_to_char = {x[0]:x[1] for x in enumerate(all_characters)}
        self.size = len(all_characters)

    def encode(self, line):
        return np.array([self.char_to_id[x] for x in line])
    
    def decode(self, tokens):
        return ''.join([self.id_to_char[x] for x in tokens])
    
vocab = Vocabulary(lines)

In [ ]:
# <go>, <eos> - специальные токены начала и конца последовательности, нужны для работы предсказаний
class Quotes(Dataset):
    def __init__(self, sentences, vocab):
        # Construct vocabulary + EOS & GO tokens
        self.sentences = sentences
        self.vocab = vocab
        self.go = self.vocab.char_to_id['<go>']
        self.eos = self.vocab.char_to_id['<eos>']
        
    def __len__(self):
        return len(self.sentences)
        
    def __getitem__(self, idx):
        tokens = self.vocab.encode(self.sentences[idx])
        _input = np.array([self.go]+tokens)
        _output = np.array(tokens+[self.eos])
        return _input, _output

In [ ]:
# Для работы pack_padded_sequence
def compose_batch(batch):
    order = np.argsort([-len(x[0]) for x in batch])
    lengths = np.array([len(x[0]) for x in batch])
    go = torch.zeros(len(batch), lengths[order[0]]).long()
    eos = torch.zeros(len(batch), lengths[order[0]]).long()
    mask = torch.zeros(len(batch), lengths[order[0]]).long()
    for i in range(len(batch)):
        current_go, current_eos = batch[i]
        go[i, :len(current_go)] = torch.tensor(current_go)
        eos[i, :len(current_eos)] = torch.tensor(current_eos)
        mask[i, :len(current_go)] = 1
    mask = mask[order]
    go = go[order]
    eos = eos[order]
    lengths = lengths[order]
    return go, eos, mask, lengths

In [ ]:
dataset = Quotes(lines, vocab)
dataloader = DataLoader(dataset, shuffle=True, batch_size=64, collate_fn=compose_batch)

а) Напишите код класса Oracle

In [ ]:
class Oracle(nn.Module):
    def __init__(self, vocabulary_size, embedding_size=128, hidden_size=256, layers=2):
        super(Oracle, self).__init__()
        self.embedding_size = embedding_size
        self.layers = layers
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocabulary_size, embedding_size)
        self.generator = nn.GRU(embedding_size, hidden_size, layers, batch_first=False)
        self.classifier = nn.Linear(hidden_size, vocabulary_size)

    # 1) Генерируем эмбединги входных токенов (из _input), пакуем в pack_padded_sequence, используя lengths
    # 2) Запускаем generator на эмбедингах, пакуем его вывод в pad_packed_sequence
    # 3) Возвращаем предикты (пока без SoftMax) через self.classifier
    def forward(self, _input, lengths):
        ## YOUR CODE HERE
        pass

    # Генерирует последовательность (когда уже обучен). Токены переводим в id с помощью vocab.char_to_id,
    # обратно - с помощью vocab.id_to_char
    # Первый токен - <go>, делаем embeding. Далее generator -- classifier -- softmax.
    # Слово сэмплируем с помощью torch.distributions.Categorical, оно становится новым токеном
    # Повторяем, пока не будет <eos> или не достигнем max_len
    # Возвращаем - сгенерированную строку
    def generate(self, vocab, max_len=100):
        pass

б) Обучите модель. Приведите примеры сгенеррованных предложений

In [ ]:
oracle = Oracle(vocab.size, embedding_size=32, hidden_size=64, layers=2)

In [ ]:
criterion = nn.CrossEntropyLoss(reduction='none')
losses = []

In [ ]:
# Чтобы снизить колебания
def moving_average(a, n=20) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
import progressbar
import tqdm

for epoch in range(1000):
    bar = progressbar.ProgressBar()
    for i, (go, eos, mask, length) in enumerate(tqdm.tqdm(dataloader, total=len(dataloader))):
        oracle.zero_grad()
        prediction = oracle(go, length).transpose(0, 1).transpose(1, 2)
        loss = (criterion(prediction, eos)*mask.float()).mean()
        loss.backward()
        [x.grad.clamp_(-1, 1) for x in oracle.parameters()]
        optimizer.step()
        losses.append(np.exp(loss.item()))
        if i % 50 == 0:
            clear_output(True)
            plt.plot(losses, label='Train')
            plt.plot(moving_average(losses), label='MA@20')
            plt.xlabel('iteration')
            plt.ylabel('perplexity')
            plt.legend()
            plt.grid()
            plt.show()

в) Обучить сеть с LSTM вместо GRU

г) Сделать разбиение train-test и нарисровать графики зависимости перплексии от числа эпох

д) Подобрать гиперпараметры, добиться лучшей перплексии чем с параметрами по умолчанию

**Спасибо за выполнение заданий!**